<a href="https://colab.research.google.com/github/dunomi/dunomi/blob/master/USA_analiza.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

ConfirmedCases_raw=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/virus_cases.csv')
USA_state_names= pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/states.csv')
USA_humidity_row=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/USA_humidity.csv?token=AO3BJXHYBDBENHN2JIXXFGC6POTP2')
USA_temperature_row=pd.read_csv('https://raw.githubusercontent.com/dunomi/dunomi/master/USA_temperature.csv?token=AO3BJXFW6K5G6GYWHX5I2GC6POTVA')

In [0]:
USA=ConfirmedCases_raw[ConfirmedCases_raw['Country/Region']=='US'].reset_index().drop(['index','Country/Region','Lat','Long'],axis=1)
#USA.sum(axis = 0, skipna = True)[1:]>10
USA=USA.head(52)
USA=USA[~USA['Province/State'].isin(['Diamond Princess','Grand Princess'])].reset_index().drop('index',axis=1)


Przygotowanie tabeli z przypadkami Corona virus

In [0]:
USAcases=pd.DataFrame(columns=['Stan','Przyrosty','Data'])

for i in range(len(USA)):
  Date=USA.loc[i][USA.loc[i][0:]!=0].index[1]
  stateName=USA.loc[i][0]
  stateValue=USA.loc[i][1:].tolist()
  stateSum=[i for i in stateValue if i>0]
  stateSumArray=np.array(stateSum[1:7])/np.array(stateSum[0:6])
  stateSumArray=np.array([round(i,2) for i in stateSumArray])
  USAcases=USAcases.append(pd.DataFrame([[stateName,stateSumArray,Date]],columns=['Stan','Przyrosty','Data']))

USAcases=USAcases.reset_index().drop('index',axis=1)[['Stan','Data','Przyrosty']]


Przygotowanie tabeli z temperaturą i wilgotnością.

USAdata - tabela z przyrostem przypadków zachorowań, stopniami temperatury oraz procentową wilgotnością.

USAdata2 - tabela z przyrostem przypadków zachorowań, przyrostem temperatury oraz przyrostem wilgotności.

In [0]:
USAweather=pd.DataFrame(columns=['Stan','Temperatura','Wilgotnosc'])

for i in range(len(USA_temperature_row)):
  stateName=USA_temperature_row.loc[i][0]
  temp=np.array(USA_temperature_row.loc[i][3:])
  wilg=np.array(USA_humidity_row.loc[i][3:])/100

  USAweather=USAweather.append(pd.DataFrame([[stateName,temp,wilg]],columns=['Stan','Temperatura','Wilgotnosc']))
  
USAdata=pd.merge(USAcases,USAweather)

In [0]:
USAweather2=pd.DataFrame(columns=['Stan','Temperatura','Wilgotnosc'])

for i in range(len(USA_temperature_row)):
  stateName=USA_temperature_row.loc[i][0]
  temp=np.array(USA_temperature_row.loc[i][4:])/np.array(USA_temperature_row.loc[i][3:-1])
  wilg=np.array(USA_humidity_row.loc[i][4:])/np.array(USA_humidity_row.loc[i][3:-1])

  USAweather2=USAweather2.append(pd.DataFrame([[stateName,temp,wilg]],columns=['Stan','Temperatura','Wilgotnosc']))

USAdata2=pd.merge(USAcases,USAweather2)

USAcumdata - tabela z danymi przygoptowanymi do wykresu dla USAdata

USAcumdata2 - tabela z danymi przygoptowanymi do wykresu dla USAdata2

In [0]:
USAcumdata=pd.DataFrame(columns=['Stan','Przyrosty','Temperatura','Wilgotnosc'])
for i in range(0,len(USAdata)):
  stateName=USAdata.loc[i][0]
  przy=np.cumprod(USAdata['Przyrosty'][i])[5]
  temp=USAdata.Temperatura[i].mean()
  wilg=USAdata.Wilgotnosc[i].mean()
  USAcumdata=USAcumdata.append(pd.DataFrame([[stateName,przy,temp,wilg]],columns=['Stan','Przyrosty','Temperatura','Wilgotnosc']))

#USAcumdata.loc[USAcumdata.Przyrosty > 50, 'Przyrosty'] = max(USAcumdata.loc[USAcumdata.Przyrosty < 50].Przyrosty)

In [0]:
USAcumdata2=pd.DataFrame(columns=['Stan','Przyrosty','Temperatura','Wilgotnosc'])
for i in range(0,len(USAdata2)):
  stateName=USAdata2.loc[i][0]
  przy=np.cumprod(USAdata2['Przyrosty'][i])[5]
  temp=USAdata2.Temperatura[i].mean()
  wilg=USAdata2.Wilgotnosc[i].mean()
  USAcumdata2=USAcumdata2.append(pd.DataFrame([[stateName,przy,temp,wilg]],columns=['Stan','Przyrosty','Temperatura','Wilgotnosc']))

USA2cumdataLess50=USAcumdata2[USAcumdata2.Przyrosty<50]
#USAcumdataMore50=USAcumdata[USAcumdata.Przyrosty>=50]

In [0]:
def plot_maker(Data,Temperatura_lub_Wilgotnosc):

  fig = make_subplots(rows=11, cols=5, shared_xaxes=False)
  for i in range(0,len(Data)):
    x=Data.loc[i]
    figX = px.scatter(x=x[Temperatura_lub_Wilgotnosc].tolist()[0:6],
                      y=x['Przyrosty'].tolist(),
                      trendline="ols")
    
    tracePoints = figX['data'][0]
    traceTradeline = figX['data'][1]

    fig.add_trace(tracePoints, row=math.floor(i/5)+1, col=i%5+1)
    fig.add_trace(traceTradeline, row=math.floor(i/5)+1, col=i%5+1)
  fig['layout'].update(height=1500, width=1000, showlegend=False, title=Temperatura_lub_Wilgotnosc)
  return fig.show()

plot_maker(USAdata,'Temperatura')
plot_maker(USAdata,'Wilgotnosc')

Wykresy

In [69]:
fig = px.scatter(x=USAcumdata['Temperatura'].tolist(), 
                 y=USAcumdata['Wilgotnosc'].tolist(),
                 size=USAcumdata['Przyrosty'].tolist(),
                 labels={'x':'Temperatura', 'y':'Wilgotnosc','size':'Przyrosty'})
fig.update_yaxes(range=[0,1])
fig.show()

'''
fig = px.scatter(x=USAcumdataMore50['Temperatura'].tolist(), 
                 y=USAcumdataMore50['Wilgotnosc'].tolist(),
                 size=USAcumdataMore50['Przyrosty'].tolist(),
                 labels={'x':'Temperatura', 'y':'Wilgotnosc','size':'Przyrosty'})
fig.show()
'''

"\nfig = px.scatter(x=USAcumdataMore50['Temperatura'].tolist(), \n                 y=USAcumdataMore50['Wilgotnosc'].tolist(),\n                 size=USAcumdataMore50['Przyrosty'].tolist(),\n                 labels={'x':'Temperatura', 'y':'Wilgotnosc','size':'Przyrosty'})\nfig.show()\n"

In [0]:
USAcumdata_map=USAcumdata.reset_index()[['Stan','Przyrosty','Temperatura','Wilgotnosc']]

new_column = pd.Series([30,30,30,30], name='Przyrosty', index=USAcumdata_map[USAcumdata_map['Przyrosty']>30].index)
USAcumdata_map.update(new_column)

In [0]:
USA_us_map=USAcumdata_map.merge(USA_state_names, left_on='Stan', right_on='State')

figSubPlot = make_subplots(rows=2, cols=4, shared_xaxes=False)

fig1 = px.choropleth(locations=USA_us_map['Abbreviation'],locationmode="USA-states", color=USA_us_map['Przyrosty'], scope="usa",title='Przyrosty')
fig2 = px.choropleth(locations=USA_us_map['Abbreviation'],locationmode="USA-states", color=USA_us_map['Temperatura'], scope="usa",title='Temperatura')
fig3 = px.choropleth(locations=USA_us_map['Abbreviation'],locationmode="USA-states", color=USA_us_map['Wilgotnosc'], scope="usa",title='Wilgotnosc')

fig1.show()
fig2.show()
fig3.show()

KORELACJE

DANE DLA PRZYROSTÓW

In [0]:
lista_temperatura=[]
lista_wilgotnosc=[]
for i in range(len(USAdata2)):
  t=USAdata2.loc[i]
  lista_temperatura.append(pd.DataFrame(data={'x':t.Przyrosty,'y':t.Temperatura}).astype(float).corr()['x'].loc['y'])
  lista_wilgotnosc.append(pd.DataFrame(data={'x':t.Przyrosty,'y':t.Wilgotnosc}).astype(float).corr()['x'].loc['y'])

lista_temperatura.sort()
lista_wilgotnosc.sort()

fig_temperatura = px.scatter(y=lista_temperatura)
fig_wilgotnosc = px.scatter(y=lista_wilgotnosc)
fig_temperatura.show()
fig_wilgotnosc.show()


In [0]:
lista_temperatura=[]
lista_wilgotnosc=[]
for i in range(len(USAdata)):
  t=USAdata.loc[i]
  lista_temperatura.append(pd.DataFrame(data={'x':t.Przyrosty,'y':t.Temperatura[0:6]}).astype(float).corr()['x'].loc['y'])
  lista_wilgotnosc.append(pd.DataFrame(data={'x':t.Przyrosty,'y':t.Wilgotnosc[0:6]}).astype(float).corr()['x'].loc['y'])

lista_temperatura.sort()
lista_wilgotnosc.sort()

fig = make_subplots(rows=1, cols=2, shared_xaxes=False,subplot_titles=("Temperatura","Wilgotność"))    

fig_temperatura = px.histogram(x=lista_temperatura,nbins=30)
fig_wilgotnosc = px.histogram(x=lista_wilgotnosc,nbins=30)

fig.add_trace(fig_temperatura['data'][0], row=1, col=1)
fig.add_trace(fig_wilgotnosc['data'][0], row=1, col=2)

fig.update_layout( yaxis_title="Ilość korelacji", bargroupgap=0.1)

fig.show()